In [1]:
import pandas as pd
import os
import sys
sys.path.append('..')
from config import get_project_root

enfermedad='PAROTIDITIS'
DATA_PATH= f"{get_project_root()}/data"
RAW_PATH = f'{DATA_PATH}/{enfermedad}_raw.csv'
INITIAL_POP = 44_044_811
infectados_filename = f'{DATA_PATH}/preprocessed/infectados_{enfermedad.lower()}.csv'
habitantes_filename = f'{DATA_PATH}/preprocessed/habitantes_{enfermedad.lower()}.csv'
vacunados_filename = f'{DATA_PATH}/preprocessed/vacunados_{enfermedad.lower()}.csv'

In [2]:
def process_raw_data(path):
    raw_df = pd.read_csv(path, na_values='NN',thousands='.', sep=',')
    raw_df.rename(columns={'AÑO':'ANO'}, inplace=True)  # Cambiar las Enies
    raw_df.set_index('ANO', inplace=True)  # Indexar por ANO
    raw_df.dropna(how='all', inplace=True)  # Remover filas vacias
    raw_df.drop(raw_df.tail(2).index,inplace=True)  # Quitar la ultima fila de la pop total
    return raw_df


In [3]:
def extract_significant_df(raw_df, batch: int):
    df = raw_df[raw_df.columns[batch*14:(batch+1)*14]].copy(deep=True)  # Obtener 12 meses y total
    df.columns=df.iloc[0]   # Primera fila son las columnas
    df.drop(df.head(1).index,inplace=True)  # remover primera fila
    df.dropna(how='all', axis=1, inplace=True)
    df = df.apply(lambda x: x.str.replace('.', '', regex=False))
    df = df.apply(lambda x: x.str.replace(',', '', regex=False))
    df = df.apply(pd.to_numeric)
    return df

In [4]:
raw = process_raw_data(RAW_PATH)
pd.set_option('display.max_columns', None)
raw

,Cantidad de infectados,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Cantidad de vacunados,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Cantidad de habitantes,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Población expuesta,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Población recuperada,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Población muerta,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82
ANO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NaN,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,Total,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,Total,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total
2018,0,0,0,0,160,7,20,16,27,16,19,0,161,426,47709,257587,777874,52176,65942,45108,44544,58574,76555,69182,55347,47442,1598040,NaN,NaN,NaN,NaN,NaN,NaN,"10,973,470",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019,0,187,5,7,12,12,6,0,21,11,10,11,183,465,51312,65032,66158,58810,59828,46475,51302,52849,73303,143902,98568,80153,847692,NaN,NaN,NaN,NaN,NaN,NaN,"11,012,606",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2020,110,21,20,1,5,2,0,2,0,0,0,4,98,263,14420,12343,12138,12617,12498,11691,9943,5697,422,9,8,5,91791,NaN,NaN,NaN,NaN,NaN,NaN,"11,047,665",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2021,48,0,3,0,3,4,0,5,4,12,5,2,58,144,2,5,2,8,1,4,1,6,51054,57821,76872,43932,229708,NaN,NaN,NaN,NaN,NaN,NaN,"11,077,856",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022,0,109,0,104,15,8,4,37,9,2,0,0,0,288,52597,103479,94272,83608,84083,90501,99570,105370,89357,NaN,NaN,NaN,802837,NaN,NaN,NaN,NaN,NaN,NaN,"11,101,945",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## Infectados

In [5]:
# Infectados son los primeros 14

infectados = extract_significant_df(raw, batch=0)

In [6]:
infectados

nan,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total
ANO,,,,,,,,,,,,,,
2018,0,0,0,0,160,7,20,16,27,16,19,0,161,426
2019,0,187,5,7,12,12,6,0,21,11,10,11,183,465
2020,110,21,20,1,5,2,0,2,0,0,0,4,98,263
2021,48,0,3,0,3,4,0,5,4,12,5,2,58,144
2022,0,109,0,104,15,8,4,37,9,2,0,0,0,288


Parece que aca los NaN serian 0 asi se condice con los totales, los ponemos y escribimos el csv correspondiente

In [7]:
infectados.fillna(0, inplace=True)
infectados

nan,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,otros,Total
ANO,,,,,,,,,,,,,,
2018,0,0,0,0,160,7,20,16,27,16,19,0,161,426
2019,0,187,5,7,12,12,6,0,21,11,10,11,183,465
2020,110,21,20,1,5,2,0,2,0,0,0,4,98,263
2021,48,0,3,0,3,4,0,5,4,12,5,2,58,144
2022,0,109,0,104,15,8,4,37,9,2,0,0,0,288


In [8]:
infectados.to_csv(infectados_filename)

## Vacunados

In [9]:
vacunados = extract_significant_df(raw, batch=1)
vacunados

nan,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGOS,SEP,OCT,NOV,DIC,Total
ANO,,,,,,,,,,,,,
2018,47709,257587,777874,52176,65942,45108,44544,58574,76555,69182.0,55347.0,47442.0,1598040
2019,51312,65032,66158,58810,59828,46475,51302,52849,73303,143902.0,98568.0,80153.0,847692
2020,14420,12343,12138,12617,12498,11691,9943,5697,422,9.0,8.0,5.0,91791
2021,2,5,2,8,1,4,1,6,51054,57821.0,76872.0,43932.0,229708
2022,52597,103479,94272,83608,84083,90501,99570,105370,89357,NaN,NaN,NaN,802837


Validemos los Nan

In [10]:
vacunados[vacunados.columns[0:12]].sum(axis=1) == vacunados.Total

ANO
2018    True
2019    True
2020    True
2021    True
2022    True
dtype: bool

In [11]:
vacunados[vacunados.columns[0:12]].sum(axis=1)

ANO
2018    1598040.0
2019     847692.0
2020      91791.0
2021     229708.0
2022     802837.0
dtype: float64

En efecto los NaN deberian ser 0

In [12]:
vacunados.fillna(0, inplace=True)
vacunados.to_csv(vacunados_filename)

## Habitantes

In [13]:
habitantes = extract_significant_df(raw, batch=2)
habitantes

nan,JUL
ANO,
2018,10973470
2019,11012606
2020,11047665
2021,11077856
2022,11101945


In [14]:
habitantes.fillna(0, inplace=True)

In [15]:
habitantes.fillna(0, inplace=True)
habitantes.to_csv(habitantes_filename)